In [1]:
import json
import os
import datetime

with open('/Users/marie/Documents/github/config.json', 'r') as f:
    config = json.load(f)

    for key in config:
        os.environ[key] = config[key]

In [2]:
import requests 
import io
import pandas as pd

/Users/marie/Documents/github/llm_env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# ! pip install langchain --upgrade

In [4]:
import langchain
langchain.__version__

'0.1.0'

In [5]:
# ! pip install langchainhub

In [35]:
langchain.debug = False

### OpenAI tools vs OpenAI functions

In [7]:
system_message = '''
You are working as a product analyst for the e-commerce company. 
Your work is very important, since your product team makes decisions based on the data you provide. So, you are extremely accurate with the numbers you provided. 
If you're not sure about the details of the request, you don't provide the answer and ask follow-up questions to have a clear understanding.
You are very helpful and try your best to answer the questions.

If you don't have enough context to answer question, you should ask user the follow-up question to get needed info. 
You don't make any assumptions about data requests. For example, if dates are not specified, you ask follow up questions. 
Always use tool if you have follow-up questions to the request.
'''

In [144]:
import random
from pydantic import BaseModel, Field
from typing import Optional
from langchain.agents import tool

class Filters(BaseModel):
    month: str = Field(description="Month of customer's activity in the format %Y-%m-%d")
    city: Optional[str] = Field(description="City of residence for customers (by default no filter)", 
                    enum = ["London", "Berlin", "Amsterdam", "Paris"])

@tool(args_schema=Filters)
def get_monthly_active_users(month: str, city: str = None) -> int:
    """Returns number of active customers for the specified month. Pass month in format %Y-%m-01.
    """
    coefs = {
        'London': 2,
        'Berlin': 1,
        'Amsterdam': 0.5,
        'Paris': 0.25
    }
    dt = datetime.datetime.strptime(month, '%Y-%m-%d')
    total = dt.year + 10*dt.month
    if city is None:
        return total
    else:
        return int(round(coefs[city]*total))
        
        
class Metrics(BaseModel):
    metric1: float = Field(description="Base metric value to calculate the difference")
    metric2: float = Field(description="New metric value that we compare with the baseline")

@tool(args_schema=Metrics)
def percentage_difference(metric1: float, metric2: float) -> float:
    """Calculates the percentage difference between metrics"""
    return (metric2 - metric1)/metric1*100

In [9]:
get_monthly_active_users.run({"month": "2023-12-01", "city": "London"})

[tool/start] [1:tool:get_monthly_active_users] Entering Tool run with input:
"{'month': '2023-12-01', 'city': 'London'}"
[tool/end] [1:tool:get_monthly_active_users] s] Exiting Tool run with output:
"4286"


4286

In [10]:
get_monthly_active_users.run({"month": "2023-12-01", "city": "Berlin"})

[tool/start] [1:tool:get_monthly_active_users] Entering Tool run with input:
"{'month': '2023-12-01', 'city': 'Berlin'}"
[tool/end] [1:tool:get_monthly_active_users] s] Exiting Tool run with output:
"2143"


2143

In [12]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [13]:
from langchain.agents import AgentType, Tool, initialize_agent, create_openai_tools_agent, create_openai_functions_agent, AgentExecutor
from langchain.schema import SystemMessage
from langchain_community.chat_models import ChatOpenAI

tools = [get_monthly_active_users, percentage_difference]

human_input_agent_tools = create_openai_tools_agent(
    llm = ChatOpenAI(temperature=0.1, model = 'gpt-4-1106-preview'),
    tools = tools, 
    prompt = prompt
)

human_input_agent_tools_executor = AgentExecutor(
    agent=human_input_agent_tools, tools=tools, 
    verbose=True, max_iterations=10, # early stopping criteria
    early_stopping_method='generate')

human_input_agent_funcs = create_openai_functions_agent(
    llm = ChatOpenAI(temperature=0.1, model = 'gpt-4-1106-preview'),
    tools = tools, 
    prompt = prompt
)

human_input_agent_funcs_executor = AgentExecutor(
    agent=human_input_agent_funcs, tools=tools, 
    verbose=True, max_iterations=10, # early stopping criteria
    early_stopping_method='generate')

/Users/marie/Documents/github/llm_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
/Users/marie/Documents/github/llm_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [32]:
human_input_agent_funcs_executor.invoke(
    {'input': 'What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?', 
     'agent_scratchpad': []})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?",
  "agent_scratchpad": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?",
  "agent_scratchpad": [],
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": "What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?",
  "agent_scratchpad": [],
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:R

{'input': 'What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?',
 'agent_scratchpad': [],
 'output': 'In December 2023, the number of customers in London was 4,286, while in Berlin it was 2,143. The percentage difference between the number of customers in London and Berlin is -50.0%, indicating that London had twice as many customers as Berlin in that month.'}

In [15]:
human_input_agent_tools_executor.invoke(
    {'input': 'What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?', 
     'agent_scratchpad': []})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?",
  "agent_scratchpad": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?",
  "agent_scratchpad": [],
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": "What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?",
  "agent_scratchpad": [],
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:R

{'input': 'What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?',
 'agent_scratchpad': [],
 'output': 'In December 2023, the number of customers in London was 4,286, and in Berlin, it was 2,143. The percentage difference between the number of customers in London and Berlin is -50.0%, indicating that London had twice as many customers as Berlin.'}

In [16]:
407 + 446 + 482 + 539

1874

In [17]:
488 + 489 + 560

1537

In [18]:
1874/1537*100.

121.92582953806115

### Human tool

In [27]:
from langchain.tools import HumanInputRun
human_tool = HumanInputRun()

In [41]:
system_message = '''
You are working as a product analyst for the e-commerce company. 
Your work is very important, since your product team makes decisions based on the data you provide. So, you are extremely accurate with the numbers you provided. 
If you're not sure about the details of the request, you don't provide the answer and ask follow-up questions to have a clear understanding.
You are very helpful and try your best to answer the questions.

If you don't have enough context to answer question, you should ask user the follow-up question to get needed info. 
You don't make any assumptions about data requests. For example, if dates are not specified, you ask follow up questions. 
Always use tool if you have follow-up questions to the request.
'''

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [28]:
human_tool.run({'query': "What's your name?"})

[tool/start] [1:tool:human] Entering Tool run with input:
"{'query': "What's your name?"}"


What's your name?


 Maria


[tool/end] [1:tool:human] [8.19s] Exiting Tool run with output:
"Maria"


'Maria'

In [24]:
human_tool.description

'You can ask a human for guidance when you think you got stuck or you are not sure what to do next. The input should be a question for the human.'

In [29]:
human_tool.args

{'query': {'title': 'Query', 'type': 'string'}}

In [31]:
human_tool.args_schema

In [38]:
tools = [get_monthly_active_users, percentage_difference, human_tool]

human_input_agent = create_openai_tools_agent(
    llm = ChatOpenAI(temperature=0.2, model = 'gpt-4-1106-preview'),
    tools = tools, 
    prompt = prompt
)

human_input_agent_executor = AgentExecutor(
    agent=human_input_agent, tools=tools, 
    verbose=True, max_iterations=10, # early stopping criteria
    early_stopping_method='generate')

/Users/marie/Documents/github/llm_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [39]:
human_input_agent_executor.invoke(
    {'input': 'What are the number of customers in London?', 
     'agent_scratchpad': []})



> Entering new AgentExecutor chain...
To provide you with the number of customers in London, I need to know the specific time period you are interested in. Are you looking for the number of monthly active users in London for a particular month, or do you need a different metric? Please provide the time frame or specify the metric you need.

> Finished chain.


{'input': 'What are the number of customers in London?',
 'agent_scratchpad': [],
 'output': 'To provide you with the number of customers in London, I need to know the specific time period you are interested in. Are you looking for the number of monthly active users in London for a particular month, or do you need a different metric? Please provide the time frame or specify the metric you need.'}

In [42]:
human_tool = HumanInputRun(
    description = 'You can use this tool to ask the user for the details related to the request. Always use this tool if you have follow-up questions. The input should be a question for the user. Be concise, polite and professional when asking the questions.'
)

In [43]:
human_tool.description

'You can use this tool to ask the user for the details related to the request. Always use this tool if you have follow-up questions. The input should be a question for the user. Be concise, polite and professional when asking the questions.'

In [44]:
tools = [get_monthly_active_users, percentage_difference, human_tool]

human_input_agent = create_openai_tools_agent(
    llm = ChatOpenAI(temperature=0.1, model = 'gpt-4-1106-preview'),
    tools = tools, 
    prompt = prompt
)

human_input_agent_executor = AgentExecutor(
    agent=human_input_agent, tools=tools, 
    verbose=True, max_iterations=10, # early stopping criteria
    early_stopping_method='generate')

/Users/marie/Documents/github/llm_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [45]:
human_input_agent_executor.invoke(
    {'input': 'What are the number of customers in London?', 
     'agent_scratchpad': []})



> Entering new AgentExecutor chain...

Invoking: `human` with `Could you please specify the time period for which you would like to know the number of customers in London?`




Could you please specify the time period for which you would like to know the number of customers in London?


 December 2023


December 2023
Invoking: `get_monthly_active_users` with `{'month': '2023-12-01', 'city': 'London'}`


4286The number of active customers in London for December 2023 is 4,286.

> Finished chain.


{'input': 'What are the number of customers in London?',
 'agent_scratchpad': [],
 'output': 'The number of active customers in London for December 2023 is 4,286.'}

### Memory basics 

In [66]:
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationTokenBufferMemory, ConversationSummaryBufferMemory
memory = ConversationBufferMemory()

In [50]:
memory.save_context(
    {"input": "Hey, how are you? How was your weekend?"}, 
    {"output": "Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic."}
)
print(memory.buffer)

Human: Hey, how are you? How was your weekend?
AI: Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic.


In [51]:
memory.save_context(
    {"input": "Could you please help me with the urgent request from our CEO. What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?"}, 
    {"output": "In December 2023, the number of customers in London was 4,286, and in Berlin, it was 2,143. The percentage difference between the number of customers in London and Berlin is -50.0%, indicating that London had twice as many customers as Berlin."}
)
print(memory.buffer)

Human: Hey, how are you? How was your weekend?
AI: Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic.
Human: Could you please help me with the urgent request from our CEO. What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?
AI: In December 2023, the number of customers in London was 4,286, and in Berlin, it was 2,143. The percentage difference between the number of customers in London and Berlin is -50.0%, indicating that London had twice as many customers as Berlin.


In [54]:
memory = ConversationBufferWindowMemory(k=1)

In [55]:
memory.save_context(
    {"input": "Hey, how are you? How was your weekend?"}, 
    {"output": "Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic."}
)
print(memory.buffer)

Human: Hey, how are you? How was your weekend?
AI: Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic.


In [56]:
memory.save_context(
    {"input": "Could you please help me with the urgent request from our CEO. What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?"}, 
    {"output": "In December 2023, the number of customers in London was 4,286, and in Berlin, it was 2,143. The percentage difference between the number of customers in London and Berlin is -50.0%, indicating that London had twice as many customers as Berlin."}
)
print(memory.buffer)

Human: Could you please help me with the urgent request from our CEO. What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?
AI: In December 2023, the number of customers in London was 4,286, and in Berlin, it was 2,143. The percentage difference between the number of customers in London and Berlin is -50.0%, indicating that London had twice as many customers as Berlin.


In [63]:
memory = ConversationTokenBufferMemory(
    llm = ChatOpenAI(temperature=0.1, model = 'gpt-4-1106-preview'), 
    max_token_limit=100)

/Users/marie/Documents/github/llm_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [64]:
memory.save_context(
    {"input": "Hey, how are you? How was your weekend?"}, 
    {"output": "Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic."}
)
print(memory.buffer)

Human: Hey, how are you? How was your weekend?
AI: Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic.


In [65]:
memory.save_context(
    {"input": "Could you please help me with the urgent request from our CEO. What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?"}, 
    {"output": "In December 2023, the number of customers in London was 4,286, and in Berlin, it was 2,143. The percentage difference between the number of customers in London and Berlin is -50.0%, indicating that London had twice as many customers as Berlin."}
)
print(memory.buffer)

AI: In December 2023, the number of customers in London was 4,286, and in Berlin, it was 2,143. The percentage difference between the number of customers in London and Berlin is -50.0%, indicating that London had twice as many customers as Berlin.


In [76]:
langchain.debug = True

In [77]:
memory = ConversationSummaryBufferMemory(
    llm = ChatOpenAI(temperature=0.1, model = 'gpt-4-1106-preview'), 
    max_token_limit=100)

/Users/marie/Documents/github/llm_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [78]:
memory.save_context(
    {"input": "Hey, how are you? How was your weekend?"}, 
    {"output": "Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic."}
)
print(memory.buffer)

[HumanMessage(content='Hey, how are you? How was your weekend?'), AIMessage(content='Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic.')]


In [79]:
print(memory.load_memory_variables({})['history'])

Human: Hey, how are you? How was your weekend?
AI: Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic.


In [80]:
memory.save_context(
    {"input": "Could you please help me with the urgent request from our CEO. What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?"}, 
    {"output": "In December 2023, the number of customers in London was 4,286, and in Berlin, it was 2,143. The percentage difference between the number of customers in London and Berlin is -50.0%, indicating that London had twice as many customers as Berlin."}
)
print(memory.buffer)

/Users/marie/Documents/github/llm_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "summary": "",
  "new_lines": "Human: Hey, how are you? How was your weekend?\nAI: Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic.\nHuman: Could you please help me with the urgent request from our CEO. What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans rea

In [82]:
print(memory.load_memory_variables({})['history'])

System: The AI had a good weekend learning about LLM agents and describes it as magical. The human requests assistance with an urgent task from the CEO, asking for the absolute numbers and percentage difference of customers in London and Berlin in December 2023.
AI: In December 2023, the number of customers in London was 4,286, and in Berlin, it was 2,143. The percentage difference between the number of customers in London and Berlin is -50.0%, indicating that London had twice as many customers as Berlin.


In [83]:
print("Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\nEND OF EXAMPLE\n\nCurrent summary:\n\n\nNew lines of conversation:\nHuman: Hey, how are you? How was your weekend?\nAI: Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic.\nHuman: Could you please help me with the urgent request from our CEO. What are the absolute numbers and the percentage difference between the number of customers in London and Berlin in December 2023?\n\nNew summary:")

Human: Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:


New lines of conversation:
Human: Hey, how are you? How was your weekend?
AI: Good morning, I had a wonder time off and spent the whole day learning about LLM agents. It works like magic.
Human: Could you please help me with the urgent request from our CEO. What are the absolute numbers and the percentage difference between

### Memory

In [84]:
langchain.debug = False

In [85]:
system_message = '''
You are working as a product analyst for the e-commerce company. 
Your work is very important, since your product team makes decisions based on the data you provide. So, you are extremely accurate with the numbers you provided. 
If you're not sure about the details of the request, you don't provide the answer and ask follow-up questions to have a clear understanding.
You are very helpful and try your best to answer the questions.

If you don't have enough context to answer question, you should ask user the follow-up question to get needed info. 
You don't make any assumptions about data requests. For example, if dates are not specified, you ask follow up questions. 
Always use tool if you have follow-up questions to the request.
'''

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

tools = [get_monthly_active_users, percentage_difference, human_tool]

human_input_agent = create_openai_tools_agent(
    llm = ChatOpenAI(temperature=0.1, model = 'gpt-4-1106-preview'),
    tools = tools, 
    prompt = prompt
)

human_input_agent_executor = AgentExecutor(
    agent=human_input_agent, tools=tools, 
    verbose=True, max_iterations=10, # early stopping criteria
    early_stopping_method='generate')

/Users/marie/Documents/github/llm_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [86]:
human_input_agent_executor.invoke(
    {'input': 'What are the number of customers in London in December 2023?', 
     'agent_scratchpad': []})



> Entering new AgentExecutor chain...

Invoking: `get_monthly_active_users` with `{'month': '2023-12-01', 'city': 'London'}`


4286The number of active customers in London in December 2023 was 4,286.

> Finished chain.


{'input': 'What are the number of customers in London in December 2023?',
 'agent_scratchpad': [],
 'output': 'The number of active customers in London in December 2023 was 4,286.'}

In [87]:
human_input_agent_executor.invoke(
    {'input': 'And what about Berlin?', 
     'agent_scratchpad': []})



> Entering new AgentExecutor chain...

Invoking: `human` with `Could you please specify the details of the data you are looking for regarding Berlin? Are you interested in monthly active users, sales figures, or another specific metric?`




Could you please specify the details of the data you are looking for regarding Berlin? Are you interested in monthly active users, sales figures, or another specific metric?


 monthly active users in December 2023


monthly active users in December 2023
Invoking: `get_monthly_active_users` with `{'month': '2023-12-01', 'city': 'Berlin'}`


2143The number of monthly active users in Berlin for December 2023 was 2,143.

> Finished chain.


{'input': 'And what about Berlin?',
 'agent_scratchpad': [],
 'output': 'The number of monthly active users in Berlin for December 2023 was 2,143.'}

In [155]:
memory = ConversationBufferMemory(
    return_messages=True, memory_key="chat_history")

memory.save_context(
    {"input": "Hey, how are you? How was your weekend?"}, 
    {"output": "Good morning, I had a wonderful time off and spent the whole day learning about LLM agents. It works like magic."}
)
print(memory.buffer)

[HumanMessage(content='Hey, how are you? How was your weekend?'), AIMessage(content='Good morning, I had a wonderful time off and spent the whole day learning about LLM agents. It works like magic.')]


In [156]:
langchain.debug = True

In [157]:
get_monthly_active_users.description

'get_monthly_active_users(month: str, city: str = None) -> int - Returns number of active customers for the specified month. Pass month in format %Y-%m-01.'

In [158]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

tools = [get_monthly_active_users, percentage_difference, human_tool]

human_input_agent = create_openai_tools_agent(
    llm = ChatOpenAI(temperature=0.1, model = 'gpt-4-1106-preview'),
    tools = tools, 
    prompt = prompt
)

human_input_agent_executor = AgentExecutor(
    agent=human_input_agent, tools=tools, 
    verbose=True, max_iterations=10, # early stopping criteria
    early_stopping_method = 'generate', memory = memory,
    handle_parsing_errors = True
)

/Users/marie/Documents/github/llm_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [159]:
human_input_agent_executor.invoke(
    {'input': 'What is the number of customers in London?'})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:<lambda>] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:<lambda>] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [1:chain:AgentExecutor > 2:ch

 December 2023


[tool/end] [1:chain:AgentExecutor > 9:tool:human] [4.26s] Exiting Tool run with output:
"December 2023"
[chain/start] [1:chain:AgentExecutor > 10:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 10:chain:RunnableSequence > 11:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 10:chain:RunnableSequence > 11:chain:RunnableAssign<agent_scratchpad> > 12:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 10:chain:RunnableSequence > 11:chain:RunnableAssign<agent_scratchpad> > 12:chain:RunnableParallel<agent_scratchpad> > 13:chain:<lambda>] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:AgentExecutor > 10:chain:RunnableSequence > 11:chain:RunnableAssign<agent_scratchpad> > 12:chain:RunnableParallel<agent_scratchpad> > 13:chain:<lambda>] s] Exiting Chain run with output:
{
  "output": [
    

{'input': 'What is the number of customers in London?',
 'chat_history': [HumanMessage(content='Hey, how are you? How was your weekend?'),
  AIMessage(content='Good morning, I had a wonderful time off and spent the whole day learning about LLM agents. It works like magic.'),
  HumanMessage(content='What is the number of customers in London?'),
  AIMessage(content='The number of active customers in London for December 2023 is 4,286.')],
 'output': 'The number of active customers in London for December 2023 is 4,286.'}

In [163]:
print("System: \nYou are working as a product analyst for the e-commerce company. \nYour work is very important, since your product team makes decisions based on the data you provide. So, you are extremely accurate with the numbers you provided. \nIf you're not sure about the details of the request, you don't provide the answer and ask follow-up questions to have a clear understanding.\nYou are very helpful and try your best to answer the questions.\n\nIf you don't have enough context to answer question, you should ask user the follow-up question to get needed info. \nYou don't make any assumptions about data requests. For example, if dates are not specified, you ask follow up questions. \nAlways use tool if you have follow-up questions to the request.\n\nHuman: Hey, how are you? How was your weekend?\nAI: Good morning, I had a wonderful time off and spent the whole day learning about LLM agents. It works like magic.\nHuman: What is the number of customers in London?")

System: 
You are working as a product analyst for the e-commerce company. 
Your work is very important, since your product team makes decisions based on the data you provide. So, you are extremely accurate with the numbers you provided. 
If you're not sure about the details of the request, you don't provide the answer and ask follow-up questions to have a clear understanding.
You are very helpful and try your best to answer the questions.

If you don't have enough context to answer question, you should ask user the follow-up question to get needed info. 
You don't make any assumptions about data requests. For example, if dates are not specified, you ask follow up questions. 
Always use tool if you have follow-up questions to the request.

Human: Hey, how are you? How was your weekend?
AI: Good morning, I had a wonderful time off and spent the whole day learning about LLM agents. It works like magic.
Human: What is the number of customers in London?


In [160]:
human_input_agent_executor.invoke(
    {'input': 'What is the number for Berlin?'})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:<lambda>] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:<lambda>] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [1:chain:AgentExecutor > 2:ch

{'input': 'What is the number for Berlin?',
 'chat_history': [HumanMessage(content='Hey, how are you? How was your weekend?'),
  AIMessage(content='Good morning, I had a wonderful time off and spent the whole day learning about LLM agents. It works like magic.'),
  HumanMessage(content='What is the number of customers in London?'),
  AIMessage(content='The number of active customers in London for December 2023 is 4,286.'),
  HumanMessage(content='What is the number for Berlin?'),
  AIMessage(content='The number of active customers in Berlin for December 2023 is 2,143.')],
 'output': 'The number of active customers in Berlin for December 2023 is 2,143.'}

In [164]:
print("System: \nYou are working as a product analyst for the e-commerce company. \nYour work is very important, since your product team makes decisions based on the data you provide. So, you are extremely accurate with the numbers you provided. \nIf you're not sure about the details of the request, you don't provide the answer and ask follow-up questions to have a clear understanding.\nYou are very helpful and try your best to answer the questions.\n\nIf you don't have enough context to answer question, you should ask user the follow-up question to get needed info. \nYou don't make any assumptions about data requests. For example, if dates are not specified, you ask follow up questions. \nAlways use tool if you have follow-up questions to the request.\n\nHuman: Hey, how are you? How was your weekend?\nAI: Good morning, I had a wonderful time off and spent the whole day learning about LLM agents. It works like magic.\nHuman: What is the number of customers in London?\nAI: The number of active customers in London for December 2023 is 4,286.\nHuman: What is the number for Berlin?")

System: 
You are working as a product analyst for the e-commerce company. 
Your work is very important, since your product team makes decisions based on the data you provide. So, you are extremely accurate with the numbers you provided. 
If you're not sure about the details of the request, you don't provide the answer and ask follow-up questions to have a clear understanding.
You are very helpful and try your best to answer the questions.

If you don't have enough context to answer question, you should ask user the follow-up question to get needed info. 
You don't make any assumptions about data requests. For example, if dates are not specified, you ask follow up questions. 
Always use tool if you have follow-up questions to the request.

Human: Hey, how are you? How was your weekend?
AI: Good morning, I had a wonderful time off and spent the whole day learning about LLM agents. It works like magic.
Human: What is the number of customers in London?
AI: The number of active customers in